In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import collections
import pickle
import gzip

from tf.app import use

In [3]:
A = use('oldassyrian:clone', checkout='clone', hoist=globals())

# Parallels

We make edges between similar lines.

When are lines similar?

If a certain distance metric is above a certain threshold.

We choose this metric:

* we reduce a line to the set of readings and graphemes in it, excluding unknown signs and ellipses.
* the similarity between two lines is the length of the intersection divided by the length of the union of their sets times 100.

# Preparation

We pre-compute all sets for all lines.

In [4]:
READABLE_TYPES = {'reading', 'grapheme', 'numeral', 'complex'}

def makeSet(l):
  if F.lnc.v(l): # comment line
    return None
  lineSet = set()
  for s in L.d(l, otype='sign'):
    if F.type.v(s) in READABLE_TYPES:
      r = F.readingr.v(s)
      if r:
        lineSet.add(r)
      g = F.graphemer.v(s)
      if g:
        lineSet.add(g)
  return lineSet

In [5]:
lines = {}

for l in F.otype.s('line'):
  lineSet = makeSet(l)
  if lineSet:
    lines[l] = lineSet
    
nLines = len(lines)
print(f'{nLines} lines')

102526 lines


# Measure

In [6]:
def sim(lSet, mSet):
  return int(round(100 * len(lSet & mSet) / len(lSet | mSet)))

# Compute all similarities

We are going to perform several hundreds of millions of comparisons, each of which is more than an elemetary operation.

Let's measure time.

In [7]:
THRESHOLD = 90

def computeSim(limit=None):
  similarity = {}

  lineNodes = sorted(lines.keys())
  nLines = len(lineNodes)

  nComparisons = nLines * (nLines - 1) // 2

  print(f'{nComparisons} comparisons to make')
  chunkSize = nComparisons // 100

  co = 0
  b = 0
  si = 0
  p = 0

  A.indent(reset=True)

  stop = False
  for i in range(nLines):
    nodeI = lineNodes[i]
    lineI = lines[nodeI]
    for j in range(i + 1, nLines):
      nodeJ = lineNodes[j]
      lineJ = lines[nodeJ]
      s = sim(lineI, lineJ)
      co += 1
      b += 1
      if b == chunkSize:
        p += 1
        A.info(f'{p:>3}% - {co:>12} comparisons and {si:>10} similarities')
        b = 0
        if limit is not None and p >= limit:
          stop = True
          break

      if s < THRESHOLD:
        continue
      similarity[(nodeI, nodeJ)] = sim(lineI, lineJ)
      si += 1
    if stop:
      break

  A.info(f'{p:>3}% - {co:>12} comparisons and {si:>10} similarities')
  return similarity

We are going to run it to 1% first and do some checks then.

In [8]:
similarity = computeSim(limit=1)

5255739075 comparisons to make
    58s   1% -     52557390 comparisons and       3747 similarities
    58s   1% -     52557390 comparisons and       3747 similarities


We check the sanity of the results.

In [9]:
print(min(similarity.values()))
print(max(similarity.values()))

100
100


In [10]:
eq = [x for x in similarity.items() if x[1] >= 100]
neq = [x for x in similarity.items() if x[1] <= 90]

In [11]:
print(len(eq))
print(len(neq))

3747
0


In [13]:
print(eq[0])
# print(neq[0])

((865347, 865350), 100)


In [14]:
A.plain(eq[0][0][0])
A.plain(eq[0][0][1])

In [15]:
# A.plain(neq[0][0][0])
# A.plain(neq[0][0][1])

Looks good.

Now the whole computation.

But if we have done this before, and nothing has changed, we load previous results from disk.

If we do not find previous results, we compute them and save the results to disk.

In [16]:
PARA_DIR = f'{A.tempDir}/parallels'

def writeResults(data, location, name):
  if not os.path.exists(location):
    os.makedirs(location, exist_ok=True)
  path = f'{location}/{name}'
  with gzip.open(path, 'wb') as f:
    pickle.dump(data, f)
  print(f'Data written to {path}')
  
def readResults(location, name):
  path = f'{location}/{name}'
  if not os.path.exists(path):
    print(f'File not found: {path}')
    return None
  with gzip.open(path, 'rb') as f:
    data = pickle.load(f)
  print(f'Data read from {path}')
  return data

In [17]:
similarity = readResults(PARA_DIR, f'sim-{A.version}.zip')
if not similarity:
  similarity = computeSim()
  writeResults(similarity, PARA_DIR, f'sim-{A.version}.zip')

File not found: /Users/dirk/github/Nino-cunei/oldassyrian/_temp/parallels/sim-0.1.zip
5255739075 comparisons to make
    57s   1% -     52557390 comparisons and       3747 similarities
 1m 55s   2% -    105114780 comparisons and      11559 similarities
 2m 53s   3% -    157672170 comparisons and      20609 similarities
 3m 50s   4% -    210229560 comparisons and      32834 similarities
 4m 47s   5% -    262786950 comparisons and      41372 similarities
 5m 45s   6% -    315344340 comparisons and      43848 similarities
 6m 43s   7% -    367901730 comparisons and      45841 similarities
 7m 41s   8% -    420459120 comparisons and      53649 similarities
 8m 39s   9% -    473016510 comparisons and      61260 similarities
 9m 37s  10% -    525573900 comparisons and      63950 similarities
10m 37s  11% -    578131290 comparisons and      66670 similarities
11m 39s  12% -    630688680 comparisons and      68581 similarities
12m 41s  13% -    683246070 comparisons and      70203 similarities

In [18]:
len(similarity)

522468

So, over half a million pairs of similar lines.

Let's find out which lines have the most correspondences.

In [19]:
parallels = {}

for (l, m) in similarity:
  parallels.setdefault(l, set()).add(m)
  parallels.setdefault(m, set()).add(l)
  
print(f'{len(parallels)} out of {nLines} lines have at least one similar line')

37824 out of 102526 lines have at least one similar line


In [20]:
rankedParallels = sorted(
  parallels.items(),
  key=lambda x: (-len(x[1]), x[0]),
)

In [21]:
for (l, paras) in rankedParallels[0:10]:
  print(f'{len(paras):>4} siblings of {l} = {T.text(l)}')

 454 siblings of 865417 = 2/2(disz) _ma-na ku3-babbar_
 454 siblings of 865670 = 1/3(disz) _ma-na ku3-babbar_
 454 siblings of 865986 = 2(disz) 1/2(disz) _ma-na ku3-babbar_
 454 siblings of 865998 = 5(disz) _ma-na ku3-babbar_
 454 siblings of 866145 = 1/2(disz) _ma-na ku3-babbar_
 454 siblings of 866243 = 1(disz) _ma-na ku3-babbar_
 454 siblings of 866325 = 2(disz) _ma-na ku3-babbar_
 454 siblings of 866381 = 8(disz) _ma-na ku3-babbar_
 454 siblings of 866431 = 1(disz) _ma-na ku3-babbar_
 454 siblings of 866798 = 1/3(disz) _ma-na ku3-babbar_


In [22]:
for (l, paras) in rankedParallels[1000:1010]:
  print(f'{len(paras):>4} siblings of {T.text(l)}')

 266 siblings of [...]-x 1/3(disz) _ma-na_ 5(disz) _gin2 ku3-babbar_
 266 siblings of [x ma]-na 7(disz) _gin2 ku3-babbar_
 266 siblings of 1(disz) _ma-na_ 5(disz) _gin2 ku3-babbar_
 266 siblings of 2/3(disz) _ma-na_ 5(disz) _gin2 ku3-babbar_
 266 siblings of 8(disz) _ma-na_ <<_gin2_>> _ku3-babbar_
 266 siblings of 2(disz) 1/2(disz) _ma-na_ 5(disz) [_gin2 ku3-babbar_]
 266 siblings of [x ma]-na 7(disz) 1/2(disz) _gin2 ku3-babbar_
 266 siblings of 1/3(disz) _ma-na_ 6(disz) _gin2 ku3-babbar_
 266 siblings of [... x ma]-na# 1(disz) _gin2 ku3-babbar_
 266 siblings of 2(disz) _ma-na_ 5(disz) _gin2 ku3-babbar_


In [23]:
for (l, paras) in rankedParallels[1130:1140]:
  print(f'{len(paras):>4} siblings of {T.text(l)}')

 198 siblings of 5(disz) _gin2 ku3-babbar_
 198 siblings of 2(disz) _gin2 ku3-babbar_
 198 siblings of 8(disz) _gin2 ku3-babbar_
 198 siblings of 1(disz) _gin2 ku3-babbar_
 198 siblings of 2/3(disz) _gin2 ku3-babbar_
 198 siblings of 1(disz) 1/2(disz) _gin2 ku3-babbar_
 198 siblings of 5(disz) _gin2 ku3-babbar_
 198 siblings of 2(disz) 2/3(disz) _gin2 ku3-babbar_
 198 siblings of 1/4(disz) _gin2 ku3-babbar_
 198 siblings of 4(disz) 1/2(disz)? _gin2 ku3-babbar_


And how many lines have just one correspondence?

We look at the tail of rankedParallels.

In [24]:
pairs = [(x, list(paras)[0]) for (x, paras) in rankedParallels if len(paras) == 1]
print(f'There are {len(pairs)} exclusively parallel pairs of lines')

There are 9412 exclusively parallel pairs of lines


Why not make an overview of exactly how wide-spread parallel lines are?

We count how many lines have how many parallels.

In [25]:
parallelCount = collections.Counter()

buckets = (2, 10, 20, 50, 100)

bucketRep = {}
prevBucket = None
for bucket in buckets:
  if prevBucket is None:
    bucketRep[bucket] = f'       n <= {bucket:>3}'
  elif bucket == buckets[-1]:
    bucketRep[bucket] = f'       n >  {bucket:>3}'
  else:
    bucketRep[bucket] = f'{prevBucket:>3} <  n <= {bucket:>3}'
  prevBucket = bucket

for (l, paras) in rankedParallels:
  clusterSize = len(paras) + 1
  if clusterSize > buckets[-1]:
    theBucket = buckets[-1]
  else:
    for bucket in buckets:
      if clusterSize <= bucket:
        theBucket = bucket
        break
  parallelCount[theBucket] += 1
  
for (bucket, amount) in sorted(
  parallelCount.items(),
  key=lambda x: (-x[0], x[1]),
):
  print(f'{amount:>4} lines have n sisters where {bucketRep[bucket]}')

5374 lines have n sisters where        n >  100
3935 lines have n sisters where  20 <  n <=  50
4676 lines have n sisters where  10 <  n <=  20
14427 lines have n sisters where   2 <  n <=  10
9412 lines have n sisters where        n <=   2


# Add parallels to the TF dataset

We can add this information to the Oldbabylonian dataset as an *edge feature*.

An edge feature links two nodes and may annotate that link with a value.

For parallels, we link each line to each of its parallel lines and we annotate that link with the similarity between
the two lines. The similarity is a percentage, and we round it to integer values.

If *n1* is similar to *n2*, then *n2* is similar to *n1*.
In order to save space, we only add such links once.

We can then use
[`E.sim.b(node)`](https://annotation.github.io/text-fabric/Api/Features/#edge-features)
to find all nodes that are parallel to node.


In [26]:
metaData = {
  '': {
    'name': 'Old Assyrian Cuneiform',
    'editor': 'various',
    'institute': 'CDL',
    'converters': 'Alba de Ridder, Martijn Kokken, Cale Johnson, Dirk Roorda',    
  },
  'sim': {
    'valueType': 'int',
    'edgeValues': True,
    'description': 'similarity between lines, as a percentage of the common material wrt the combined material',
  },
}

In [27]:
simData = {}
for ((f, t), d) in similarity.items():
  simData.setdefault(f, {})[t] = d

In [28]:
ghBase = os.path.expanduser('~/github')
subdir = 'parallels'
path = f'{A.context.org}/{A.context.repo}/{subdir}/tf'
location = f'{ghBase}/{path}'
module = A.version

In [29]:
TF.save(edgeFeatures=dict(sim=simData), metaData=metaData, location=location, module=module)

  0.00s Exporting 0 node and 1 edge and 0 config features to ~/github/Nino-cunei/oldassyrian/parallels/tf/0.1:
   |     0.53s T sim                  to ~/github/Nino-cunei/oldassyrian/parallels/tf/0.1
  0.53s Exported 0 node features and 1 edge features and 0 config features to ~/github/Nino-cunei/oldassyrian/parallels/tf/0.1


True

# Turn the parallels feature into a module

The new `sim` feature is a big data feature. You do not want to load it all the time.

Here we show how to turn it into a module, so that users can easily load it in a Jupyter notebook or in the TF browser.

In [31]:
%%bash
text-fabric-zip 'Nino-cunei/oldassyrian/parallels/tf'

True
Create release data for Nino-cunei/oldassyrian/parallels/tf
Found 1 versions
zip files end up in ~/Downloads/Nino-cunei-release/oldassyrian
zipping Nino-cunei/oldassyrian     0.1 with   1 features ==> parallels-tf-0.1.zip


I have added this file to a new release of the Oldbabylonian Github repo.

# Use the parallels module

We load the Oldbabylonian corpus again, but now with the parallels module.

In [32]:
A = use('oldassyrian:clone', hoist=globals(), checkout="clone",
    mod='Nino-cunei/oldassyrian/parallels/tf:clone',
)

   |     0.51s T sim                  from ~/github/Nino-cunei/oldassyrian/parallels/tf/0.1


Lo and behold: you see the parallels module listed with one feature: `sim`. It is in *italics*, which indicates
it is an edge feature.

We just do a quick check here and in another notebook we study parallels a bit more, using the feature `sim`.

We count how many similar pairs their are, and how many 100% similar pairs there are.

In [33]:
query = '''
line
-sim> line
'''
results = A.search(query)

  1.39s 522468 results


In [34]:
query = '''
line
-sim=100> line
'''
results = A.search(query)

  1.36s 518204 results


Remarkably, most of the pairs are 100 percent similar. Let's show just a few:

In [35]:
A.table(results, start=1, end=10, withNodes=True)

n,p,line,line
1,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,8653501/2(disz) 1/6(disz) 1(u) 5(disz) 7(disz) 7(disz) 7(disz)
2,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,8900132(u) 1(disz) x [...]
3,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,892244[x] x 1(u) 2(disz) x [...]
4,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,8924451(u) 5(disz) x [...]
5,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,9057011(u) 5(disz) 5(disz) 2(disz)
6,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,9057041(u) 7(disz) 1/2(disz) 3(disz)
7,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,9135461(u) 5(disz) 7(disz) 7(disz) 7(disz) 7(disz) x x
8,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,9135521(u) 5(disz) 7(disz) 7(disz) 7(disz)
9,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,9323747(u) 3(disz) 4(u) 5(disz)
10,P361245 obverse:1,8653477(disz) 7(disz) 1(u) 5(disz) 7(disz) 7(disz)# 7(disz)# 7(disz)#,9413791(u) 3(disz) [...]


There is also a lower level way to work with edge features.

We can list all edges going out from a reference node.
What we see is tuple of pairs: the target node and the similarity between the reference node and that target node.

In [39]:
refNode = 865350

E.sim.f(refNode)

((890013, 100),
 (892244, 100),
 (892445, 100),
 (905701, 100),
 (905704, 100),
 (913546, 100),
 (913552, 100),
 (932374, 100),
 (941379, 100),
 (947448, 100),
 (954502, 100),
 (960802, 100))

Likewise, we can observe the nodes that target the reference node:

In [40]:
E.sim.t(refNode)

((865347, 100),)

Both sets of nodes are similar to the reference node and it is inconvenient to use both `.f()` and `.t()` to get the similar lines.

But there is another way:

In [41]:
E.sim.b(refNode)

((865347, 100),
 (890013, 100),
 (892244, 100),
 (892445, 100),
 (905701, 100),
 (905704, 100),
 (913546, 100),
 (913552, 100),
 (932374, 100),
 (941379, 100),
 (947448, 100),
 (954502, 100),
 (960802, 100))

Let's make sure that `.b()` gives the combination of `.f()` and `.t()`.

In [42]:
f = {x[0] for x in E.sim.f(refNode)}
b = {x[0] for x in E.sim.b(refNode)}
t = {x[0] for x in E.sim.t(refNode)}

# are f and t disjoint ?

print(f'the intersection of f and t is {f & t}')

# is b the union of f and t ?

print(f't | f = b ? {f | t == b}')

the intersection of f and t is set()
t | f = b ? True
